# Bibliotecas necessárias

In [ ]:
# pip install -r 'requirements.txt'

# Bibliotecas necessarias
from pysus.online_data.CNES import download #para baixar dados do datasus
import pandas as pd #maninpulacao de dados
from pandas.tseries.offsets import MonthEnd #transformacao de datas
import geopandas as gpd #manipulacao de dados espaciais
import numpy as np #para funcoes matematicas
import statsmodels.api as sm # analise de varianca
from statsmodels.formula.api import ols # regressao linear
import seaborn as sns #para graficos
import matplotlib.pyplot as plt #para graficos
import os # acesso a arquivos do sistema
import splot #plotar graficos espaciais
import mapclassify as mc #personalisar graficos espaciais
from libpysal.weights.contiguity import Queen #matriz de vizinhanda (I-MORAN)
from esda import Moran, Moran_Local # Indices
from splot.esda import plot_moran, moran_scatterplot, lisa_cluster, plot_local_autocorrelation #graficos espaciais


# Funcao para deixar graficos mais bonitos
def eixo_seta():
    ax = plt.gca()
    fig = plt.gcf()
    for side in ['right','top']:
        ax.spines[side].set_visible(False)

# Carregamento e transformação nos dados

 Os dados utilizados são de domínio público e estão disponíveis no [servidor FTP do DataSUS](https://datasus.saude.gov.br/transferencia-de-arquivos/). Além disso, para fins de análises complementares foram utilizados também as seguintes fontes de dados:

- malhas geográficas dos municípios goianienses. Fonte: [Instituto Brasileiro de Geografia e Estatística](https://www.ibge.gov.br/geociencias/organizacao-do-territorio/malhas-territoriais/15774-malhas.html?=&t=downloads)
- dados de contaminação e mortes da Covid-19. Fonte: [COVID-19 Data Repository by the Center for Systems Science and Engineering (CSSE) at Johns Hopkins University](https://github.com/wcota/covid19br)


## Tratamento da base principal: leitos no estado de Goiás

In [ ]:
### DADOS DOS LEITOS DATASUS
dfs = {}
for ano in [2020,2021,2022]:
    for mes in range(1,13):
        nome = f"{ano}_{mes}"
        if nome == '2022_12':
            pass
        else:
            dfs[nome] = download('LT', state='GO', year=ano, month=mes)

leitos = pd.concat(dfs.values())

#Apenas colunas de interesse ao estudo
tipos = {
    "CODUFMUN": str,
    "COMPETEN": str,
    "QT_EXIST": "int64",
    "QT_CONTR": "int64",
    "QT_SUS": "int64",
    "QT_NSUS": "int64"
}

leitos = leitos[list(tipos.keys())]

for coluna, tipo in tipos.items():
    leitos[coluna] = leitos[coluna].astype(tipo)


leitos.rename(columns={"CODUFMUN": "CD_MUN"}, inplace=True) #padroniza nome de coluna

# Padronizando competencia para o ultimo dia do mês
leitos["COMPETEN"] = pd.to_datetime(leitos["COMPETEN"], format="%Y%m") + MonthEnd(0)
leitos['COMPETEN'] = leitos['COMPETEN'].dt.date 
leitos['COMPETEN'] = leitos['COMPETEN'].astype(str) #transforma novamente em string para realizar o merge
leitos.head()

## Dados de contaminação Covid-19

In [ ]:
# DADOS COVID

## Dados precisam ser baixados e extraidos antes da execucao deste bloco
## link: https://github.com/wcota/covid19br/blob/master/cases-brazil-cities-time_2020.csv.gz
## link: https://github.com/wcota/covid19br/blob/master/cases-brazil-cities-time_2021.csv.gz
## link: https://github.com/wcota/covid19br/blob/master/cases-brazil-cities-time.csv.gz

cols = [
    "date",
    "ibgeID",
    "newDeaths",
    "deaths",
    "newCases",
    "totalCases",
    "deaths_per_100k_inhabitants",
    "totalCases_per_100k_inhabitants",
    "deaths_by_totalCases",
]

# Carregando arquivos baixados e extraidos
cv_files = [cvd for cvd in os.listdir() if "cases-brazil" in cvd]
bancos_covid = {}
for banco in cv_files:
    bancos_covid[banco] = pd.read_csv(banco, usecols=cols)

# Concatenando arquivos
covid = pd.concat(bancos_covid.values())

# Tratando competencia
covid["date"] = pd.to_datetime(covid["date"]) + MonthEnd(0)  # ultimo dia do mes
covid["date"] = covid["date"].dt.date
covid["date"] = covid["date"].astype(str)

covid["ibgeID"] = covid["ibgeID"].astype(str)

covid = covid.groupby(["ibgeID", "date"]).sum().reset_index()
covid["deaths_by_totalCases"] = (
    covid["deaths_per_100k_inhabitants"] / covid["totalCases_per_100k_inhabitants"]
)

# Renomeando
covid.rename(
    columns={
        "ibgeID": "CD_MUN",
        "deaths": "total_mortes",
        "deaths_per_100k_inhabitants": "mortes_per_100k",
        "totalCases_per_100k_inhabitants": "casos_per_100k",
        "deaths_by_totalCases": "prob_momrte",
        "date": "COMPETEN",
    },
    inplace=True,
)

covid.head()

# Carregamendo da malha geográfica dos municípios goianos

In [ ]:
### MALHA GEOGRAFICA

# Disponível em: https://www.ibge.gov.br/geociencias/organizacao-do-territorio/malhas-territoriais/15774-malhas.html?=&t=downloads
go = gpd.read_file("GO_malha/GO_Municipios_2021.shp")

# Unindo bases

In [ ]:
# Unindo malha com dados do covid pela chave Codigo Municipio
df = go.merge(covid, on='CD_MUN', how='left')
df.head()

# Unindo bases 
df['CD_MUN'] = df['CD_MUN'].apply(lambda x: x[:-1]) #remove o digito para realizar o merge
df = df.merge(leitos, on=['COMPETEN', 'CD_MUN'], how='right')
df['COMPETEN'] = df['COMPETEN'].astype('datetime64') # transforma em data
df.tail()


# Análise exploratória de dados

In [ ]:
fig, ax = plt.subplots(figsize=[16, 5])
df_agg_competencia = df[
    [
        "COMPETEN",
        "newDeaths",
        "total_mortes",
        "newCases",
        "totalCases",
        "mortes_per_100k",
        "casos_per_100k",
        "prob_momrte",
        "QT_EXIST",
        "QT_CONTR",
        "QT_SUS",
        "QT_NSUS",
    ]
].copy()

# Agrupando valores por competência
df_agg_competencia = df_agg_competencia.groupby("COMPETEN").sum().reset_index()

# Tranformando data wide para data long para facilitar o plot
df_agg_competencia = df_agg_competencia.melt(id_vars="COMPETEN", var_name="serie", value_name="total")

# Definindo filtro
inicio_pandemia = np.datetime64("2020-02-29")

# Para localizar e plotar ponto no grafico
filt = df_agg_competencia["serie"].isin(["QT_SUS", "QT_NSUS"])
ymax = df_agg_competencia.loc[((df_agg_competencia["COMPETEN"] == inicio_pandemia) & (filt)), "total"]

# Grafico
sns.lineplot(
    x="COMPETEN", y="total", style="serie", color="black", data=df_agg_competencia[filt], ax=ax
)
plt.axvline(x=inicio_pandemia, ymax=0.9, ls="--", color="red")
ax.set_xlabel("")
plt.legend(["SUS", "NÃO SUS"])
plt.annotate("Primeiro caso \n registrado", xy=(inicio_pandemia, 13200))
ax.set_ylabel("Quantidade de leitos SUS")
eixo_seta()


In [ ]:
fig, ax = plt.subplots(1,1, figsize=[16,5])

df_dist = df[['COMPETEN', 'QT_SUS', 'QT_NSUS']].groupby('COMPETEN').sum().reset_index()
df_dist = df_dist.melt(id_vars='COMPETEN', value_name='total', var_name='serie')
df_dist['ANO'] = df_dist['COMPETEN'].dt.year 
df_dist['serie'].replace({'QT_SUS':'SUS', 'QT_NSUS':'Não SUS'}, inplace=True)

sns.boxplot(x='ANO', y='total', hue='serie', data=df_dist, ax=ax)

eixo_seta()
plt.legend(title='')
ax.set_ylabel("Quantidade de leitos SUS")
ax.set_xlabel("")
plt.savefig('leitos_agg_boxplot.png',
    dpi=200,
    bbox_inches='tight',
    pad_inches=.0,
)

In [ ]:
df_agg_contaminacao = df[['COMPETEN',
       'newCases', 'totalCases']].groupby('COMPETEN').sum().reset_index()

df_agg_contaminacao['newCases'] = df_agg_contaminacao['newCases'] / 1000000
fig, ax = plt.subplots(figsize=[16,5])
# sns.lineplot(x='COMPETEN', y='totalCases', data=df_agg_contaminacao)
sns.lineplot(x='COMPETEN', y='newCases', color='black', data=df_agg_contaminacao)
plt.xlabel("")
plt.ylabel("Casos novos registrados em milhões")
eixo_seta()
plt.savefig('novos_casos.png',
    dpi=200,
    bbox_inches='tight',
    pad_inches=.0,
)

## Análise de variância

In [ ]:
# Analise de variancia

## Selecionando dados
df_anova = (
    df[["COMPETEN", "QT_SUS", "QT_NSUS"]]
    .groupby("COMPETEN")
    .sum()
    .reset_index()
    .melt(id_vars="COMPETEN", value_name="total", var_name="serie")
)

df_anova["COMPETEN"] = df_anova["COMPETEN"].dt.date
df_anova["COMPETEN"] = df_anova["COMPETEN"].astype(str)

model = ols("total ~ C(COMPETEN) + C(serie)", data=df_anova).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
anova_table

# Teste Tukey para identificar diferenças
# from statsmodels.stats.multicomp import pairwise_tukeyhsd

# tukey = pairwise_tukeyhsd(
#     endog=df_anova['total'], groups=df_anova['COMPETEN'], alpha=.05
# )
# tukey.summary()

In [ ]:
# Correlação 
corr = df[['COMPETEN','newCases', 'QT_SUS', 'QT_NSUS']].groupby('COMPETEN').sum().reset_index()
corr.replace({np.nan:0}, inplace=True)
corr.set_index('COMPETEN', inplace=True)
corr.corr().round(4)

# Análise Exploratória de Dados Espaciais (AEDE)

In [ ]:
# Segregação dos períodos
segregacao = {
    "Julho/2020": np.datetime64("2020-07-31"),
    "Julho/2021": np.datetime64("2021-07-31"),
    "Julho/2022": np.datetime64("2022-07-31"),
}

fig, ax = plt.subplots(1, 3, figsize=[16, 5])

for i, nome in enumerate(segregacao.items()):

    # Filtra periodo 
    filt = df["COMPETEN"] == nome[1]
    if i < 2:
        # Plota mapa sem legenda
        df[filt].plot(
            column="QT_EXIST",
            cmap="Blues",
            ax=ax[i],
            scheme="quantiles",
            legend=False,
            missing_kwds={"color": "lightgrey"},
        )
    else:
        # Plota mapa com legenda
        df[filt].plot(
            column="QT_EXIST",
            cmap="Blues",
            ax=ax[i],
            scheme="quantiles",
            legend=True,
            missing_kwds={"color": "lightgrey"},
        )
    
    # Corrige titulo e remove eixos
    ax[i].set_title(nome[0])
    ax[i].set_axis_off()

leg = ax[2].get_legend()
leg.set_bbox_to_anchor((1.3, 0.4))


# Mapas LISA 

In [ ]:
# subselecionando malha
go_lisa = go.copy()
go_lisa['CD_MUN'] = go_lisa['CD_MUN'].apply(lambda x: x[:-1])
go_lisa.head()

In [ ]:
# Subselecionando dados e criando novo geo dataframe de interesse
lisa_df = df[['CD_MUN', 'COMPETEN', 'QT_EXIST']].groupby(['CD_MUN', 'COMPETEN']).sum().reset_index()

filt = lisa_df['COMPETEN']==np.datetime64("2020-07-31")
lisa_df = go_lisa.merge(lisa_df[filt], on='CD_MUN', how='right')
w = Queen.from_dataframe(lisa_df, geom_col='geometry')
w.transform = 'r'

In [ ]:
# INDICE MORAN GLOBAL
y = lisa_df['QT_EXIST'].values
moran = Moran(y, w)
moran.I 

In [ ]:
# valor-p 
moran.p_sim #REJEITA H0

In [ ]:
# Personalização de mapas LISA
import matplotlib.lines as mlines
aa = mlines.Line2D([0], [0], marker='o', color='w', markerfacecolor='#d7191c', markersize=12)
ab = mlines.Line2D([0], [0], marker='o', color='w', markerfacecolor='#fdae61', markersize=12)
ba = mlines.Line2D([0], [0], marker='o', color='w', markerfacecolor='#abd9e9', markersize=12)
bb = mlines.Line2D([0], [0], marker='o', color='w', markerfacecolor='#2c7bb6', markersize=12)
ns = mlines.Line2D([0], [0], marker='o', color='w', markerfacecolor='lightgrey', markersize=12)


#legenda=['High-High', 'High-Low',  'Low-High','Low-Low', 'Not significant']
legenda=['Alto-Alto', 'Alto-Baixo',  'Baixo-Alto', 'Baixo-Baixo', 'Não significativo']
handles = [aa, ab, ba, bb,  ns]


fig, ax = plt.subplots(1, 3, figsize=(16,5))

segregacao = {
"Julho/2020": np.datetime64('2020-07-31'), 
"Julho/2021": np.datetime64('2021-07-31'),
"Julho/2022": np.datetime64('2022-07-31')
}

lisa_df = df[['CD_MUN', 'COMPETEN', 'QT_EXIST']].groupby(['CD_MUN', 'COMPETEN']).sum().reset_index()

for i, mes in enumerate(segregacao.items()):

    filt = lisa_df['COMPETEN']==mes[1]
    lisa_for_w = go_lisa.merge(lisa_df[filt], on='CD_MUN', how='right')
    w = Queen.from_dataframe(lisa_for_w, geom_col='geometry')
    y = lisa_for_w['QT_EXIST'].values
    w.transform = 'r'

    lisa = Moran_Local(y, w)
    lisa_cluster(lisa, lisa_for_w, p=0.05, ax=ax[i], legend=False)
    ax[i].set_title(mes[0])
    ax[i].set_axis_off()
    ax[i].legend(handles=handles, labels=legenda, loc='upper right', bbox_to_anchor=(1.5, 1))